# <선형 회귀분석의 기초>

- 바이어스 오그멘테이션 : 상수항 추가하기

- OLS  
    : `Scikit-Learn` 패키지를 사용하여 선형회귀분석을 하는 경우에는 `linear_model` 서브 패키지의 `LinearRegression` 클래스를 사용한다.  
    : `statsmodels` 패키지에서는 `OLS` 클래스를 사용하여 선형회귀분석을 실시한다. 


# <확률론적 선형회귀모형>

- 부트스트래핑 : 기존 데이터 재표본화 - 이번 프로젝트에서 하지 않음

### 확률론적 선형회귀모형의 가정
- 가정1. 오차의 분포에 대한 가정  
    1-1. 선형 정규 분포 가정  
    1-2. 외생성(Exogeneity) 가정  
    1-3. 조건부 독립 가정  
  

- 가정 2. 독립 변수에 대한 가정


- 잔차의 분포 : 잔차는 정규분포를 따른다. -> 정규성검정  
    Q-Q plot  
    ```
    sp.stats.probplot(result.resid, plot=plt)
    ```  

    omni_normtest  
    ```
    test = sms.omni_normtest(result.resid)
    for xi in zip(['Chi^2', 'P-value'], test):
        print("%-12s: %6.3f" % xi)
    ```  

    jarque_bera
    ```
    test = sms.jarque_bera(result.resid)
    for xi in zip(['Jarque-Bera', 'P-value', 'Skew', 'Kurtosis'], test):
        print("%-12s: %6.3f" % xi)
    ```
    
- 잔차의 분포2 : 잔차의 기댓값도 $x$와 상관없이 0이어야 한다.

### 회귀계수의 표준오차
$w$에 대한 표준오차. 모수 오차는 기댓값이 0이고 표준 오차를 분산으로 가지는 스튜던트-t 분포를 따른다.

### 단일 계수 t-검정 (Single Coefficient t-test)

$$
\dfrac{\hat{w}_i - w_i}{se_i} \sim t_{N-K} \;\; (i=0, \ldots, K-1)
$$

StatsModels summary 메서드가 출력하는 회귀분석 보고서에서 std err로 표시된 열이 모형계수의 표준오차, t로 표시된 열이 단일 계수 t-검정의 검정 통계량, 그리고 P>|t|로 표시된 열이 유의확률을 뜻한다.

### Loss-of-Fit 검정 (회귀 분석 F-검정)

$$
H_0 : w_0  = w_1 = \cdots = w_{K-1} = 0
$$

# <레버리지와 아웃라이어>

#### 레버리지 확인 
레버리지 값은 `RegressionResults` 클래스의 `get_influence` 메서드로 구할 수 있다.  

선형회귀 결과에서 get_influence 메서드를 호출하면 영향도 정보 객체를 구할 수 있다. 이 객체는 hat_matrix_diag 속성으로 레버리지 벡터를 가지고 있다.

```
influence = result.get_influence()
hat = influence.hat_matrix_diag
```

#### 레버리지의 영향
: 레버리지가 큰 데이터를 포함하는 경우 vs 포함하지 않는 경우 확인

#### 아웃라이어
- 표준화 잔차가 큰 값을 제거  
- 잔차는 `RegressionResult` 객체의 `resid` 속성에 있다.  
- 표준화 잔차는 `resid_pearson` 속성에 있다. 보통 표준화 잔차가 2~4보다 크면 아웃라이어로 본다.

#### Cook's Distance
잔차와 레버리지를 동시에 보기위한 기준으로는 Cook's Distance가 있다. 다음과 같이 정의되는 값으로 레버리지가 커지거나 잔차의 크기가 커지면 Cook's Distance 값이 커진다.

모든 데이터의 레버리지와 잔차를 동시에 보려면 `plot_leverage_resid2` 명령을 사용한다. 이 명령은 x축으로 표준화 잔차의 제곱을 표시하고 y축으로 레버리지값을 표시한다. 데이터 아이디가 표시된 데이터들이 레버리지가 큰 아웃라이어이다.

`influence_plot` 명령을 사용하면 Cook's distance를 버블 크기로 표시한다.

```
sm.graphics.influence_plot(result, plot_alpha=0.3)
plt.show()
```

# <R 스타일 모형 정의>

patsy 패키지는 회귀분석 전처리를 위한 패키지로 데이터프레임을 가공하여 인코딩, 변환 등을 쉽게 해주는 기능을 제공한다.

patsy 패키지의 `dmatrix`라는 명령을 사용하면 실험설계행렬(experiment design matrix)을 간단히 만들수 있다. `dmatrix`에 다음과 같이 모형 정의 문자열 `formula`와 원데이터 `data`을 입력하면 `formula`에서 지정한 대로 변환된 데이터 `data_transformed`를 출력한다.

```
dmatrix("x1")
```

* dmatrix 연산자 있음. 참고

### 변환
dmatrix에서는 일반적인 수학 변환(transform)도 가능하다. numpy 함수 뿐 아니라 사용자 정의 함수와 다음과 같은 patsy 전용 함수도 사용할 수 있다.
- center(x): 평균 제거
- standardize(x): 평균 제거 및 표준편차로 스케일링
- scale(x): standardize(x) 과 같음

### 변수 보호 , 다항선형회귀
함수를 사용한 변수 변환 이외에도 모형 정의 문자열 자체내에 연산기호를 넣어 연산한 값을 만드는 것도 가능하다. 이 때에는 모형정의 연산자와 혼동되지 않도록 I() 연산자를 추가해야 한다.
```
dmatrix("I(x1 + x2)")
```

### 카테고리 변수 인코딩
데이터로 문자열이 오는 경우에는 카테고리 값으로 인정하여 One-Hot-Encoding 방식의 인코딩을 하게 된다.

### OLS.from_formula 메서드

선형회귀분석을 위한 `OLS` 클래스에는 모형 정의 문자열을 사용할 수 있는 `from_formula`라는 메서드가 있다. 이 메서드를 쓰면 사용자가 데이터 행렬을 직접 정의하지 않고 모형 정의 문자열만으로 선형회귀모형을 만드는 것이 가능하다.

- 다음 두 모형은 동일하다.

    직접 데이터 행렬을 만드는 경우
    ```
    dfy = df.iloc[:, -1]
    dfX = sm.add_constant(df.iloc[:, :-1])
    model1 = sm.OLS(dfy, dfX)
    ```
    모형 정의 문자열을 사용하는 경우
    ```
    model2 = sm.OLS.from_formula("y ~ x1 + x2", data=df)
    ```
    
    ```
    model1.fit().summary()
    model2.fit().summary()
    ```

# <입력변수가 카테고리값인 경우>

### 카테고리 독립 변수와 더미 변수

- 더미변수 예 1
    $$
    y = f(x_1)
    $$


- 더미변수 예 2
    $$
    y = f(x_1) + wx_2
    $$
    
    
- 더미변수 예 3
    $$
    y = f(x_1) + w(x_1)x_2
    $$

# <분산 분석과 모형 성능>

### 분산 분석

statsmodels 에서는 다음과 같이 `anova_lm` 명령을 사용하여 분산 분석표를 출력할 수 있다. 다만 이 명령을 사용하기 위해서는 모형을 `from_formula` 메서드로 생성하여야 한다.

`anova_lm` 명령으로 구한 F 검정통계량과 유의확률은 모형 `summary` 명령으로 구한 `F-statistic` 및 `Prob (F-statistic)`과 일치한다.

```
sm.stats.anova_lm(result)
print(result.summary())
```

### F 검정을 사용한 변수 중요도 비교
F검정을 이용해서 변수 중요도를 비교해볼 수 있다.


F 검정은 각 독립변수의 중요도를 비교하기 위해 사용할 수 있다. 방법은 전체 모형과 각 변수 하나만을 뺀 모형들의 성능을 비교하는 것이다. 이는 간접적으로 각 독립 변수의 영향력을 측정하는 것과 같다.

`anova_lm` 명령에서는 `typ=2`로 지정하여 각 모형에서의 F 검정을 계산할 수 있다.

```
model_full = sm.OLS.from_formula("MEDV ~ CRIM + ZN + INDUS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + CHAS", data=df_boston)
model_reduced = sm.OLS.from_formula("MEDV ~ ZN + INDUS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + CHAS", data=df_boston)

sm.stats.anova_lm(model_reduced.fit(), model_full.fit())
```

```
model_boston = sm.OLS.from_formula("MEDV ~ CRIM + ZN + INDUS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT + CHAS", data=df_boston)
result_boston = model_boston.fit()
sm.stats.anova_lm(result_boston, typ=2)
```

### 분산 분석을 이용한 카테고리 값의 영향 분석
```
model_chas = sm.OLS.from_formula("MEDV ~ C(CHAS) + 0", data=df_boston)
sm.stats.anova_lm(model_chas.fit())
```

### 조정결정계수

$R^2_{adj}$ : $R^2$ 값을 보정해 준 것.

### 정보량 규준
AIC, BIC <- log likelihood에 마이너스 붙여서 변형  
값이 작을 수록 올바른 모형에 가깝다.

# <회귀분석 결과의 진단>

### 잔차 정규성
잔차와 x의 관계를 그렸을 때, 특정한 상관관계를 가진다면 올바른 모형이 아니다.  
**--> 그렇 어떻게 해야하는지??????????????????**

### 이분산성
실제 데이터는 독립 변수 값의 크기가 커지면 종속 변수 값의 분산도 커지는 이분산성(heteroskedastic) 문제가 발생한다. -> log변환

### 자기상관계수

잔차가 서로 독립이 아니면 시계열 모형을 써야함.

# <스케일링과 변수 변환>

### 조건수 Condition No.

조건수가 크면 약간의 오차만 있어도 해가 전혀 다른 값을 가진다. 따라서 조건수가 크면 회귀분석을 사용한 예측값도 오차가 커지게 된다.

회귀분석에서 조건수가 커지는 경우는 크게 두 가지가 있다.

1. 변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우. 이 경우에는 스케일링(scaling)으로 해결한다.
2. 다중 공선성 즉, 상관관계가 큰 독립 변수들이 있는 경우, 이 경우에는 변수 선택이나 PCA를 사용한 차원 축소 등으로 해결한다.

### 스케일링
StatsModels에서는 모형지정 문자열에서 `scale` 명령을 사용하여 스케일링을 할 수 있다.

```
model2 = sm.OLS.from_formula("MEDV ~ "
                             "scale(CRIM) + scale(ZN) + scale(INDUS) + scale(NOX) + scale(RM) + scale(AGE) + "
                             "scale(DIS) + scale(RAD) + scale(TAX) + scale(PTRATIO) + scale(B) + scale(LSTAT) + CHAS", 
                             data=df)
result2 = model2.fit()
print(result2.summary()
```

### 변수변환

다음과 같은 경우에는 로그 함수 혹은 제곱근 함수 등을 사용하여 변환된 변수를 사용하면 회귀 성능이 향상될 수도 있다.

독립 변수나 종속 변수가 심하게 한쪽으로 치우친 분포를 보이는 경우
독립 변수와 종속 변수간의 관계가 곱셈 혹은 나눗셉으로 연결된 경우
종속 변수와 예측치가 비선형 관계를 보이는 경우

```
model3 = sm.OLS.from_formula("np.log(MEDV) ~ "
                             "scale(CRIM) + scale(ZN) + scale(INDUS) + "
                             "scale(NOX) + scale(RM) + scale(AGE) + "
                             "scale(np.log(DIS)) + scale(RAD) + scale(TAX) + "
                             "scale(np.log(PTRATIO)) ",
                             data=df)
result3 = model3.fit()
print(result3.summary())
```

# <다중공선성>

독립변수의 일부가 다른 독립변수의 조합으로 표현될 수 있는 경우

### VIF
StatsModels에서는 `variance_inflation_factor` 명령으로 VIF를 계산한다.  

VIF Factor 가 큰 변수를 제거하고  
-> OLS를 돌려본다.  
-> 이전보다 얼마나 개선이 됐는지 $R^2$ 값을 확인해본다.

```
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(dfX0.values, i) for i in range(dfX0.shape[1])]
vif["features"] = dfX0.columns
vif
```



# <다항회귀와 과최적화>

### 비선형 데이터 
종속변수와 독립변수의 관계가 선형이 아니고 비선형.  
선형회귀 말고 다항회귀 사용 -> 선형 기저함수 모형

### 다항회귀
다항식을 사용

$$
y = w_0 + w_1x + w_2x^2  + \ldots  + w_M x^M
$$

StatsModels에서는 `OLS` 클래스의 `from_formula` 메서드를 사용하여 다항회귀를 할 수 있다.
- 1차
    ```
    sm.OLS.from_formula("y ~ x", data=df).fit().summary()
    ```
- 2차
    ```
    sm.OLS.from_formula("y ~ x + I(x**2)", data=df).fit().summary())
    ```
    
### 과최적화
과최적화는

- 독립 변수 데이터 갯수에 비해 모형 모수의 수가 과도하게 크거나
- 독립 변수 데이터가 서로 독립이 아닌 경우에 발생한다.

이러한 상황에서는 같은 조건에 대해 답이 복수개 존재할 수 있기 때문이다.

과최적화가 문제가 되는 이유는 다음과 같다.

- 트레이닝에 사용되지 않은 새로운 독립 변수 값을 입력하면 오차가 커진다. (cross-validation 오차)
- 샘플이 조금만 변화해도 가중치 계수의 값이 크게 달라진다. (추정의 부정확함)

새로운 x 데이터가 들어올 때 오차가 커지는 것을 볼 수 있다.



# <교차검증>

### 모형검증

### 교차검증
아래 두가지를 구해야함.
- 평균성능
- 성능분산

### Cross-Validation

#### K-fold CV
데이터 셋을 K개의 sub-set로 분리
```
from sklearn.model_selection import KFold

cv = KFold(n_splits=3, shuffle=True, random_state=0)

for train_index, test_index in cv.split(X):
    print("test index :", test_index)
    print("." * 80 )        
    print("train index:", train_index)
    print("=" * 80 )
```


#### Leave-One-Out (LOO)
하나만 뺀 샘플
```
from sklearn.model_selection import LeaveOneOut

cv = LeaveOneOut()

for train_index, test_index in cv.split(X):
    print("test X:", X[test_index])
    print("." * 80 )        
    print("test y:", y[test_index])
    print("=" * 80 ) 
```

#### ShuffleSplit
중복된 데이터를 허용
```
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=5, test_size=.5, random_state=0)

for train_index, test_index in cv.split(X):
    print("test X:\n", X[test_index])
    print("=" * 20 )        
```

### 교차 평가 시행
실제 시행은 `cross_val_score()`

- cross_val_score(estimator, X, y=None, scoring=None, cv=None)
    - cross validation iterator cv를 이용하여 X, y data 를 분할하고 estimator에 넣어서 scoring metric을 구하는 과정을 반복

- 인수
    - estimator : ‘fit’메서드가 제공되는 모형
    - X : 배열
        - 독립 변수 데이터
    - y : 배열
        - 종속 변수 데이터
    - scoring : 문자열
        - 성능 검증에 사용할 함수
    - cv : Cross Validator
        - None 이면 디폴트인 3-폴드 CV
        - 숫자 K 이면 K-폴드 CV
        - Cross Validator 클래스 객체

- 반환값
    - scores
        - 계산된 성능 값의 리스트
        
```
from sklearn.model_selection import cross_val_score

cross_val_score(model, X, y, scoring="r2", cv=cv)
```

# <정규화 선형회귀>

과최적화를 막는 방법이다. Regularized Method, Penalized Method, Contrained Least Squares 이라고도 불리운다.

### Ridge 회귀 모형

### Lasso 회귀 모형

### Elastic Net 회귀 모형

### statsmodels의 정규화 회귀 모형
statsmodels 패키지는 OLS 선형 회귀 모형 클래스의 `fit_regularized` 메서드를 사용하여 Elastic Net 모형 계수를 구할 수 있다.

### Scikit-Learn의 정규화 회귀 모형
Scikit-Learn 패키지에서는 정규화 회귀 모형을 위한 Ridge, Lasso, ElasticNet 이라는 별도의 클래스를 제공한다. 

### 정규화 모형의 장점

### Ridge 모형과 Lasso 모형의 차이

# <최적 정규화>

어떤 모형구조가 최적일 것인가??

### 다항 회귀의 차수 결정

### 정규화 하이퍼 모수 최적화